In [1]:
import requests
import time 
import pandas as pd

In [2]:
url = 'https://www.reddit.com/r/disneyparks.json'

In [3]:
headers = {'user-agent': 'Jessica 0.1'}

In [4]:
res = requests.get(url,headers=headers)

In [5]:
res.status_code

200

In [6]:
json = res.json()

In [7]:
sorted(json.keys())

['data', 'kind']

In [8]:
json['data'].keys()

dict_keys(['after', 'dist', 'modhash', 'geo_filter', 'children', 'before'])

In [9]:
len(json['data']['children'])

25

In [10]:
disney_df = pd.DataFrame(json['data']['children']) 

In [11]:
disney_df.head(3)

,kind,data
0,t3,"{'approved_at_utc': None, 'subreddit': 'disney..."
1,t3,"{'approved_at_utc': None, 'subreddit': 'disney..."
2,t3,"{'approved_at_utc': None, 'subreddit': 'disney..."


In [12]:
json['data']['after']

't3_1bvmiwy'

In [13]:
[post['data']['name'] for post in json['data']['children']]

['t3_1byh1bg',
 't3_1bynok8',
 't3_1byyb6u',
 't3_1by8xf8',
 't3_1byhfjz',
 't3_1bybmys',
 't3_1byhkrp',
 't3_1bx5n21',
 't3_1bwy591',
 't3_1bx3b8g',
 't3_1bxf7d2',
 't3_1bwooj0',
 't3_1bwmjg8',
 't3_1bwk467',
 't3_1bxvnrb',
 't3_1bwiyzl',
 't3_1bw3njw',
 't3_1bwt2qg',
 't3_1bwxg12',
 't3_1bvzhyf',
 't3_1bvvakt',
 't3_1bw2e5h',
 't3_1bv70fw',
 't3_1bw1o0y',
 't3_1bvmiwy']

In [14]:
param = {'after': 't3_1bfjjvr'}

In [15]:
requests.get(url,params=param,headers=headers)

<Response [200]>

### Key Exploration to filter out the important keys

In [16]:
json['data']['children'][1]['data'].keys()

dict_keys(['approved_at_utc', 'subreddit', 'selftext', 'author_fullname', 'saved', 'mod_reason_title', 'gilded', 'clicked', 'title', 'link_flair_richtext', 'subreddit_name_prefixed', 'hidden', 'pwls', 'link_flair_css_class', 'downs', 'thumbnail_height', 'top_awarded_type', 'hide_score', 'name', 'quarantine', 'link_flair_text_color', 'upvote_ratio', 'author_flair_background_color', 'subreddit_type', 'ups', 'total_awards_received', 'media_embed', 'thumbnail_width', 'author_flair_template_id', 'is_original_content', 'user_reports', 'secure_media', 'is_reddit_media_domain', 'is_meta', 'category', 'secure_media_embed', 'link_flair_text', 'can_mod_post', 'score', 'approved_by', 'is_created_from_ads_ui', 'author_premium', 'thumbnail', 'edited', 'author_flair_css_class', 'author_flair_richtext', 'gildings', 'content_categories', 'is_self', 'mod_note', 'created', 'link_flair_type', 'wls', 'removed_by_category', 'banned_by', 'author_flair_type', 'domain', 'allow_live_comments', 'selftext_html', 

In [17]:
info = ['name','subreddit','title','selftext','upvote_ratio','num_comments','created_utc']
for item in info:
    print(item,':',json['data']['children'][1]['data'][item])

name : t3_1bynok8
subreddit : disneyparks
title : Core Memory 
selftext : A few weeks ago my fiance (34F) and I (29M) went to Florida on a quick trip, initially to finish a tattoo appointment that was started 3 years ago. We were in Orlando for 4 days and figured that we should at least go to one of the parks while we’re here. She’s never been to Disney and I haven’t been back since I was maybe around 10 years old. She’s not huge on disney or thrill rides, nor was she sold on the idea of “Disney Magic”. 

Since we were unsure of which park to go to, we decided to go to Disney Springs, since it’s free &amp; while we were there the magic had already begun. She saw her first pair of ears and immediately started looking at all other things disney. We then decided to just buy a 2 day ticket for Magic Kingdom to catch the fireworks and Epcot for drinking around the world. 

From Disney Springs we head over to Magic Kingdom and we drive up to the employee to pay for parking  and she asks if w

### Data Collection

In [18]:
# Only need these information 'name','subreddit','title','selftext','upvote_ratio','num_comments','created_utc'

def get_subreddit(url):

    posts_data = []
    after = None
    for i in range(40):
            if after ==None:
                params={}
            else:
                params = {'after':after}
            url=url
            res = requests.get(url, params=params, headers=headers)
            if res.status_code == 200:
                the_json = res.json()
                for post in the_json['data']['children']:
                    post_data = {
                        'name': post['data']['name'],
                        'subreddit': post['data']['subreddit'],
                        'title': post['data']['title'],
                        'selftext': post['data']['selftext'],
                        'upvote_ratio': post['data']['upvote_ratio'],
                        'num_comments': post['data']['num_comments'],
                        'created_utc': post['data']['created_utc']
                    }
                    posts_data.append(post_data)
                after = the_json['data']['after']
            else:
                print(res.status_code)
                break
            time.sleep(1)
        
    df = pd.DataFrame(posts_data)
    return df

In [19]:
disneyland_url = 'https://www.reddit.com/r/disneyparks/new.json'
disneyland_df = get_subreddit(disneyland_url)
disneyland_df.tail(3)

,name,subreddit,title,selftext,upvote_ratio,num_comments,created_utc
995,t3_16kmmzx,disneyparks,Disney Tokyo,Hello. I was wondering if they offer any packa...,0.57,3,1.694910e+09
996,t3_16km0ul,disneyparks,Why don’t they let the villains roam through MK?,It just seems like it’s lacking fun characters...,0.69,4,1.694908e+09
997,t3_16k57lf,disneyparks,Epcot Next Weekend,Planning on a quick trip to Disney Epcot for t...,0.67,2,1.694864e+09


In [20]:
disneyland_df.shape

(998, 7)

In [21]:
disneyland_df['name'].nunique()

998

In [22]:
universalstudios_url = 'https://www.reddit.com/r/universalstudios/new.json'
universalstudios_df = get_subreddit(universalstudios_url)
universalstudios_df.head(3)

,name,subreddit,title,selftext,upvote_ratio,num_comments,created_utc
0,t3_1byfu2u,universalstudios,What will the extra price be to upgrade 4 day ...,I would like to get an annual pass for two par...,1.00,1,1.712525e+09
1,t3_1byahv5,universalstudios,Universal City Studios - Universal Studios Hol...,,1.00,0,1.712512e+09
2,t3_1bxw0mx,universalstudios,Pass help,Hey everyone ! I have an urgent question! My d...,0.75,11,1.712466e+09


In [23]:
universalstudios_df.shape

(991, 7)

In [24]:
universalstudios_df['name'].nunique()

991

In [25]:
disneyland_df.to_csv('../data/disneyland.csv')
universalstudios_df.to_csv('../data/universalstudios.csv')